<a href="https://colab.research.google.com/github/moushumipriya/Norwegian--Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install Dependencies and Download Llama Model

# Install all necessary libraries
!pip install numpy==1.26.4
!pip install faiss-cpu
!pip install transformers sentence-transformers PyPDF2 sentencepiece accelerate
!pip install llama-cpp-python

import os
from google.colab import files

# Create necessary folders
if not os.path.exists("./models"):
    os.makedirs("./models")
if not os.path.exists("./embeddings"):
    os.makedirs("./embeddings")

# Define the model download URL
model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf?download=true"
model_filename = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"

print(f"Downloading Llama model from the link...")
# Use wget to download the model directly to the models folder
!wget -O ./models/{model_filename} "{model_url}"

print("Llama model download complete.")

In [ ]:
# Cell 2: Upload PDF File

from google.colab import files
import os

print("Please upload your PDF file:")
uploaded_pdf = files.upload()

if uploaded_pdf:
    pdf_filename = list(uploaded_pdf.keys())[0]
    print(f"File '{pdf_filename}' uploaded successfully.")

    if pdf_filename != 'data.pdf':
        os.rename(pdf_filename, 'data.pdf')
        print(f"File has been renamed to 'data.pdf'.")
else:
    print("Error: No file was uploaded. Please try again.")

if os.path.exists('data.pdf'):
    print("Verification successful: 'data.pdf' is in the current directory.")
else:
    print("Verification failed: 'data.pdf' could not be found after upload.")

In [ ]:
# Cell 3: Main Chatbot Functions and Configuration

import PyPDF2
import numpy as np
import pickle
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, pipeline
from llama_cpp import Llama
import torch
import os

# --- Configuration ---
PROJECT_ROOT = "./"
PDF_FILE_PATH = os.path.join(PROJECT_ROOT, "data.pdf")
FAISS_INDEX_PATH = os.path.join(PROJECT_ROOT, "embeddings/faiss_index.pkl")
LLM_MODEL_PATH = os.path.join(PROJECT_ROOT, "models", "mistral-7b-instruct-v0.1.Q4_K_M.gguf")

CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
TOP_K_RETRIEVAL = 5
LLM_N_GPU_LAYERS = -1
LLM_N_CTX = 4096
LLM_TEMPERATURE = 0.7

# --- Global Models ---
embedding_model = None
tokenizer = None
translator_en_to_no = None
translator_no_to_en = None
translator_en_to_bn = None
translator_bn_to_en = None
translator_no_to_bn = None
translator_bn_to_no = None
llm_model = None

# --- Text Processing and Embedding Functions ---
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
        print(f"Extracted {len(text)} characters from PDF.")
    except Exception as e:
        print(f"Error extracting text from PDF '{pdf_path}': {e}")
        text = ""
    return text

def chunk_text(text, chunk_size=CHUNK_SIZE, overlap_size=CHUNK_OVERLAP):
    if not text or not tokenizer: return []
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    if chunk_size <= overlap_size: chunk_size = overlap_size + 100
    for i in range(0, len(tokens), chunk_size - overlap_size):
        chunk_tokens = tokens[i : i + chunk_size]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    print(f"Chunked text into {len(chunks)} chunks.")
    return chunks

def generate_embeddings(text_chunks):
    if not embedding_model or not text_chunks: return []
    print(f"Generating embeddings for {len(text_chunks)} chunks...")
    embeddings = embedding_model.encode(text_chunks, convert_to_tensor=False)
    print("Embeddings generated.")
    return embeddings.astype('float32')

def create_faiss_index(embeddings, chunks, index_path=FAISS_INDEX_PATH):
    if not embeddings.tolist(): return
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    os.makedirs(os.path.dirname(index_path), exist_ok=True)
    with open(index_path, "wb") as f:
        pickle.dump({"index": index, "chunks": chunks}, f)
    print(f"FAISS index created and saved to {index_path}.")

def load_faiss_index(index_path=FAISS_INDEX_PATH):
    try:
        with open(index_path, "rb") as f:
            data = pickle.load(f)
            index = data["index"]
            chunks = data["chunks"]
        print(f"FAISS index loaded from {index_path}.")
        return index, chunks
    except FileNotFoundError:
        print(f"FAISS index not found at {index_path}.")
        return None, None
    except Exception as e:
        print(f"Error loading FAISS index: {e}")
        return None, None

def search_faiss_index(query_embedding, index, chunks, k=TOP_K_RETRIEVAL):
    if index is None or chunks is None or query_embedding is None: return []
    query_embedding_np = np.array([query_embedding]).astype('float32')
    D, I = index.search(query_embedding_np, k)
    retrieved_chunks = [chunks[i] for i in I[0] if i < len(chunks)]
    print(f"Retrieved {len(retrieved_chunks)} chunks.")
    return retrieved_chunks


def load_translation_models():
    global translator_en_to_no, translator_no_to_en, translator_en_to_bn, translator_bn_to_en, translator_no_to_bn, translator_bn_to_no

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_name = "facebook/nllb-200-distilled-600M"
    print(f"Loading translation models on {device}...")

    try:
        translator_en_to_no = pipeline("translation", model=model_name, src_lang="eng_Latn", tgt_lang="nob_Latn", device=0 if device == "cuda" else -1)
        translator_no_to_en = pipeline("translation", model=model_name, src_lang="nob_Latn", tgt_lang="eng_Latn", device=0 if device == "cuda" else -1)
        translator_en_to_bn = pipeline("translation", model=model_name, src_lang="eng_Latn", tgt_lang="ben_Beng", device=0 if device == "cuda" else -1)
        translator_bn_to_en = pipeline("translation", model=model_name, src_lang="ben_Beng", tgt_lang="eng_Latn", device=0 if device == "cuda" else -1)
        translator_no_to_bn = pipeline("translation", model=model_name, src_lang="nob_Latn", tgt_lang="ben_Beng", device=0 if device == "cuda" else -1)
        translator_bn_to_no = pipeline("translation", model=model_name, src_lang="ben_Beng", tgt_lang="nob_Latn", device=0 if device == "cuda" else -1)
        print("All translation models loaded successfully.")
    except Exception as e:
        print(f"Warning: Could not load some translation models. {e}")
        translator_en_to_no = None
        translator_no_to_en = None
        translator_en_to_bn = None
        translator_bn_to_en = None
        translator_no_to_bn = None
        translator_bn_to_no = None

def get_translator(src_lang, tgt_lang):
    if src_lang == "bn" and tgt_lang == "en": return translator_bn_to_en
    if src_lang == "en" and tgt_lang == "bn": return translator_en_to_bn
    if src_lang == "no" and tgt_lang == "en": return translator_no_to_en
    if src_lang == "en" and tgt_lang == "no": return translator_en_to_no
    if src_lang == "bn" and tgt_lang == "no": return translator_bn_to_no
    if src_lang == "no" and tgt_lang == "bn": return translator_no_to_bn
    return None

def translate(text, src_lang, tgt_lang):
    if not text: return ""
    if src_lang == tgt_lang: return text

    translator = get_translator(src_lang, tgt_lang)
    if translator:
        try:
            return translator(text, src_lang=get_lang_code(src_lang), tgt_lang=get_lang_code(tgt_lang))[0]['translation_text']
        except Exception as e:
            print(f"Error translating from {src_lang} to {tgt_lang}: {e}")
            return text
    else:
        print(f"Translator from {src_lang} to {tgt_lang} not loaded.")
        return text

def get_lang_code(lang):
    if lang == "en": return "eng_Latn"
    if lang == "bn": return "ben_Beng"
    if lang == "no": return "nob_Latn"
    return "eng_Latn"

def load_llama_model():
    global llm_model
    if not os.path.exists(LLM_MODEL_PATH):
        print(f"Error: Llama model not found at {LLM_MODEL_PATH}. Please ensure it's uploaded.")
        return None
    try:
        print(f"Loading Llama model from {LLM_MODEL_PATH}...")
        llm_model = Llama(
            model_path=LLM_MODEL_PATH,
            n_gpu_layers=LLM_N_GPU_LAYERS,
            n_ctx=LLM_N_CTX,
            verbose=False
        )
        print("Llama model loaded successfully.")
    except Exception as e:
        print(f"Error loading Llama model: {e}")
        llm_model = None

def get_llm_response(prompt):
    if llm_model is None:
        return "Sorry, the language model is not available."
    try:
        output = llm_model.create_completion(
            prompt,
            max_tokens=500,
            temperature=LLM_TEMPERATURE,
            stop=["\nSpørsmål:", "\nContext:", "\n"],
        )
        return output["choices"][0]["text"].strip()
    except Exception as e:
        print(f"Error getting LLM response: {e}")
        return "Sorry, there was an error processing your request with the language model."

def chat_with_bot(query, input_lang):
    global faiss_index, text_chunks

    translated_query_en = translate(query, input_lang, "en")
    print(f"Translated query to EN for retrieval: {translated_query_en}")

    query_embedding_en = generate_embeddings([translated_query_en])
    if not query_embedding_en.tolist():
        return "Sorry, I could not process your query.", "দুঃখিত, আমি আপনার প্রশ্নটি প্রক্রিয়া করতে পারিনি।", "Beklager, jeg kunne ikke behandle spørsmålet ditt."

    retrieved_chunks = search_faiss_index(query_embedding_en[0], faiss_index, text_chunks)
    context = "\n\n".join(retrieved_chunks)

    if not context:
        return "I could not find relevant information in the document.", "নথিতে আমি প্রাসঙ্গিক তথ্য খুঁজে পাইনি।", "Jeg fant ingen relevant informasjon i dokumentet."

    llm_prompt_en = f"""You are a helpful assistant that answers questions based on the provided information.
    Use only the provided context to answer the question. If the answer is not in the context, say that you do not know.
    Provide a concise and direct answer.

    Context:
    {context}

    Question: {translated_query_en}
    Answer:
    """

    llm_answer_en = get_llm_response(llm_prompt_en)
    print(f"LLM Answer (EN):\n{llm_answer_en}")

    llm_answer_bn = translate(llm_answer_en, "en", "bn")
    llm_answer_no = translate(llm_answer_en, "en", "no")

    print(f"LLM Answer (BN):\n{llm_answer_bn}")
    print(f"LLM Answer (NO):\n{llm_answer_no}")

    return llm_answer_en, llm_answer_bn, llm_answer_no

In [ ]:
# Cell 4 (New): Chatbot Initialization

def main_init():
    global embedding_model, tokenizer, faiss_index, text_chunks

    # 1. Load Embedding model and Tokenizer
    print("Loading embedding model and tokenizer...")
    embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    print("Embedding model and tokenizer loaded.")

    # 2. Load Translation models
    load_translation_models()

    # 3. Load Llama model
    load_llama_model()

    # 4. Prepare document embeddings
    faiss_index, text_chunks = load_faiss_index(FAISS_INDEX_PATH)
    if faiss_index is None or text_chunks is None:
        print("\nFAISS index not found. Generating new one...")
        raw_text = extract_text_from_pdf(PDF_FILE_PATH)
        if raw_text:
            chunks = chunk_text(raw_text)
            embeddings = generate_embeddings(chunks)
            if embeddings.tolist():
                create_faiss_index(embeddings, chunks, FAISS_INDEX_PATH)
                faiss_index, text_chunks = load_faiss_index(FAISS_INDEX_PATH)
            else:
                print("Failed to generate embeddings. Chatbot will not function.")
                return
        else:
            print("Failed to extract text from PDF. Chatbot will not function.")
            return

    print("\nInitialization complete! Chatbot is ready.")

# Run the initialization function
main_init()

In [ ]:
# Cell 5 (New): Input-Output Loop (Revised)

# Start the chat loop
while True:
    user_query = input("\nSelect your language (English/Bangla/Norwegian) or type (exit) to quit: ").lower()
    if user_query == "exit":
        print("Chatbot is shutting down.")
        break

    input_lang = user_query
    user_query = input("Enter your question: ")

    if input_lang in ["bn", "en", "no"]:
        answer_en, answer_bn, answer_no = chat_with_bot(user_query, input_lang)
        print("\n--- Chatbot Response ---")
        print(f"Original Query ({input_lang}): {user_query}")
        print(f"Answer (English): {answer_en}")
        print(f"Answer (Bengali): {answer_bn}")
        print(f"Answer (Norwegian): {answer_no}")
    else:
        print("Invalid input. Please use bn, en, or no.")